In [1]:
import os
import sys
import pandas as pd
import numpy as np
sys.path.append(os.path.dirname((os.path.abspath(''))))

from src.pipeline import Pipeline
from src.preprocessing.preprocessing import removal, tokenization, stemming

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\D073999\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\D073999\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\D073999\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
collection = pd.read_csv('../data/TREC_Passage/collection.tsv', sep='\t', names=["pID", "Passage"], header=None)
queries = pd.read_csv('../data/TREC_Passage/queries.dev.tsv', sep='\t', names=["qID", "Query"], header=None)

In [3]:
collection = collection.loc[0:10000, :]
queries = queries.loc[0:10000, :]

In [4]:
collection.head()

,pID,Passage
0,0,The presence of communication amid scientific ...
1,1,The Manhattan Project and its atomic bomb help...
2,2,Essay on The Manhattan Project - The Manhattan...
3,3,The Manhattan Project was the name for a proje...
4,4,versions of each volume as well as complementa...


In [5]:
collection['preprocessed'] = collection.Passage.progress_apply(lambda text: np.array(
            stemming(
                removal(
                    tokenization(text)
                ))))

100%|██████████| 10001/10001 [00:23<00:00, 428.60it/s]


In [6]:
queries['preprocessed'] = queries.Query.progress_apply(lambda text: np.array(
            stemming(
                removal(
                    tokenization(text)
                ))))

100%|██████████| 10001/10001 [00:14<00:00, 676.35it/s]


In [7]:
from flair.embeddings import ELMoEmbeddings
from flair.data import Sentence
from nltk.tokenize.treebank import TreebankWordDetokenizer
from tqdm import tqdm
import torch
import os

class Elmo(object):
    def __init__(self):
        # init embedding
        self.embedding = ELMoEmbeddings()

    def transform(self, X: pd.Series, store: str = None):

        elmo_list = []
        for line in tqdm(X):
            detokenized = TreebankWordDetokenizer().detokenize(line)
            sentence = Sentence(detokenized)
            self.embedding.embed(sentence)
            input = torch.empty(sentence[0].embedding.size())
            tokens_per_sentence = torch.zeros_like(input)
            for token in sentence:
                tokens_per_sentence = torch.add(tokens_per_sentence, token.embedding)
            elmo_list.append(tokens_per_sentence)
            
            if store is not None:
                check_path_exists(os.path.dirname(store))
                save(elmo_list, store)
        return elmo_list

In [ ]:
elmo = Elmo()

collection['elmo'] = elmo.transform(
    collection.Passage)
queries['elmo'] = elmo.transform(queries.Query)

  2%|▏         | 180/10001 [10:19<13:00:49,  4.77s/it]

In [ ]:
    def create_glove_feature(self, path: str = 'data/embeddings'):
        embeddings = np.array(collection['elmo'])
        embeddings_queries = np.array(queries['elmo'])

        features['glove_cosine'] = self.features.progress_apply(lambda qrel:
                                                                     cosine_similarity_score(embeddings_queries[
                                                                                                 self.queries[
                                                                                                     self.queries[
                                                                                                         'qID'] == qrel.qID].index],
                                                                                             embeddings[self.collection[
                                                                                                 self.collection[
                                                                                                     'pID'] == qrel.pID].index]),
                                                                     axis=1)
        features['glove_euclidean'] = self.features.progress_apply(lambda qrel:
                                                                        euclidean_distance_score(embeddings_queries[
                                                                                                     self.queries[
                                                                                                         self.queries[
                                                                                                             'qID'] == qrel.qID].index],
                                                                                                 embeddings[
                                                                                                     self.collection[
                                                                                                         self.collection[
                                                                                                             'pID'] == qrel.pID].index]),
                                                                        axis=1)
        features['glove_manhattan'] = self.features.progress_apply(lambda qrel:
                                                                        manhattan_distance_score(embeddings_queries[
                                                                                                     self.queries[
                                                                                                         self.queries[
                                                                                                             'qID'] == qrel.qID].index],
                                                                                                 embeddings[
                                                                                                     self.collection[
                                                                                                         self.collection[
                                                                                                             'pID'] == qrel.pID].index]),
                                                                        axis=1)